In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import custom_module as cm
import optimizer_module as om

In [2]:
#############################################################################
####################### DATA PREPROCESSING #################################
###########################################################################

In [3]:
# Loading the dataset
data = pd.read_csv("combined_data.csv", parse_dates=True)
#data = data["2016":]

In [4]:
# Load this file for saving time
# Saving the data file so we can reload with the features made again to reduce time
# data.to_csv('final.csv') 
data = pd.read_csv("final.csv", parse_dates=True, index_col=0)

In [5]:
data

,RRP5MIN,RESIDUAL_DEMAND,AVG_PRICE,DIFF_PRICE,hour,weekday,month,business hour,public holiday
SETTLEMENTDATE,,,,,,,,,
2016-01-01 00:00:00,4.096091,1593.93,5.046193,-0.486409,0,0,1,0,1
2016-01-01 00:05:00,5.257498,1557.06,5.102443,1.161408,0,0,1,0,1
2016-01-01 00:10:00,5.415838,1510.10,5.087579,0.158340,0,0,1,0,1
2016-01-01 00:15:00,5.415838,1474.70,5.087579,0.000000,0,0,1,0,1
2016-01-01 00:20:00,3.609315,1464.90,4.937838,-1.806523,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
2019-06-30 23:40:00,5.998145,546.95,4.556324,-0.136612,23,0,6,0,0
2019-06-30 23:45:00,5.961220,560.19,4.778101,-0.036925,23,0,6,0,0
2019-06-30 23:50:00,5.066510,559.87,4.907920,-0.894710,23,0,6,0,0


In [6]:
# Setting the data to index
data["SETTLEMENTDATE"] = pd.to_datetime(data["SETTLEMENTDATE"])
data.index = data["SETTLEMENTDATE"]
data.drop(columns="SETTLEMENTDATE", inplace=True)

KeyError: 'SETTLEMENTDATE'

In [ ]:
# replace outliers by outlier threshold
data = cm.replace_outliers(data, 'RRP5MIN', 4)

In [ ]:
# Seperating the test dataset for testing purposes in evaluation
X_test = data["2019-01-01":"2019-06-30"].copy()
X_test = X_test["RRP5MIN"]

In [ ]:
####################### SOME CHARTS OF THE DATA IN HAND ####################
###########################################################################

In [ ]:
# Plotting the curve Price
cm.plot_chart(data["RRP5MIN"].loc["2018-02-20":"2018-02-21"], legend=True)

In [ ]:
# Plotting the curve Residual Demand
# Looks stationary
cm.plot_chart(data["RESIDUAL_DEMAND"].loc["2019"], legend=True)

In [ ]:
# Testing whether the series is Stationary via Dickey Fuller Test
# TESTING FOR PRICE
cm.is_Stationary(data.loc["2018"]["RRP5MIN"])
# TESTING FOR RESIDUAL DEMAND
cm.is_Stationary(data.loc["2018"]["RESIDUAL_DEMAND"])

In [ ]:
# This function plots the ACF and PACF plots to find useful lags for time series
cm.serial_corr(data.loc["2018"]['RRP5MIN'], lags=1000)

In [ ]:
#### ADD CORRELATION PLOT HERE WITH OTHER VARIABLES #####

In [ ]:
# ADD MORE IF YOU WANT

In [ ]:
#############################################################################
####################### FEATURES PREPROCESSING #############################
###########################################################################

In [ ]:
# Avg Price of last 1 hour i.e. 12 data points at 5 minutes granularity
data["AVG_PRICE"] = pd.DataFrame(cm.average_hours(data["RRP5MIN"]))

In [ ]:
# Differencing the average price and creating a differenced price variable
data["AVG_PRICE"] = cm.period_difference(data["AVG_PRICE"])
data["DIFF_PRICE"] = cm.period_difference(data["RRP5MIN"])

In [ ]:
############################################################################

In [ ]:
# Generate 'hour', 'weekday' and 'month' features
data['hour'] = 0
data['weekday'] = 0
data['month'] = 0
for i in range(len(data)):
    position = data.index[i]
    data['hour'][i] = position.hour
    data['weekday'][i] = position.weekday()
    data['month'][i] = position.month

In [ ]:
# MAKING FEATURES
# Generate 'business hour' feature. 7am-7pm business hours
data["business hour"] = 0
for i in range(len(data)):
    position = data.index[i]
    hour = position.hour
    if ((hour > 7 and hour < 12) or (hour > 14 and hour < 19)):
        data["business hour"][i] = 2
    elif (hour >= 12 and hour <= 14):
        data["business hour"][i] = 1
    else:
        data["business hour"][i] = 0

In [ ]:
# Generate 'weekend' feature
for i in range(len(data)):
    position = data.index[i]
    weekday = position.weekday()
    if (weekday == 6):
        data['weekday'][i] = 2
    elif (weekday == 5):
        data['weekday'][i] = 1
    else:
        data['weekday'][i] = 0

In [ ]:
from datetime import date
import holidays

In [ ]:
aus_holidays = holidays.CountryHoliday('AUS', prov='SA')

In [ ]:
data["public holiday"] = 0
for i in range(len(data)):
    if (data.index[i] in aus_holidays):
        data["public holiday"][i] = 1

In [ ]:
##### SAVE FILE HERE #####

In [ ]:
#############################################################################
########### PREPARING DATA FOR KERAS TO PROCESS PREPROCESSING ##############
###########################################################################

In [ ]:
################################## 2 MODELS #######################################################
######### 1st for processing Categorical Data for Regression via Multi-Layer Perceptron #########
########################### 2nd for processing Time Series via LSTM ##############################
################################################################################################

In [ ]:
# Scaling the RRP between 0 and 1 as required by the NN
features = ['RESIDUAL_DEMAND', 'AVG_PRICE', 'DIFF_PRICE']
feature_scaler = MinMaxScaler()
for i in features:
    data[i] = feature_scaler.fit_transform(pd.DataFrame(data[i]))

In [ ]:
# scale price data to 0-1 range
label_scaler = MinMaxScaler()
data['RRP5MIN'] = label_scaler.fit_transform(data['RRP5MIN'].values.reshape(-1, 1))

In [ ]:
train = data['2016-12-25 00:00:00':].copy()

In [ ]:
train

In [ ]:
# include time lags of timeseries data for last day i.e. 288 data points at 5 minutes granularity
# Also 80 lags of same day previous week

# Creating Daily lags
for i in range(1,201):
    train["price_l_{}".format(i)] = train["DIFF_PRICE"].shift(i)
    train["demand_l_{}".format(i)] = train["RESIDUAL_DEMAND"].shift(i)
    train["avgPrice_l_{}".format(i)] = train["AVG_PRICE"].shift(i)
    

# Creating Daily lags
for i in range(255,325):
    train["price_l_{}".format(i)] = train["DIFF_PRICE"].shift(i)
    train["demand_l_{}".format(i)] = train["RESIDUAL_DEMAND"].shift(i)
    train["avgPrice_l_{}".format(i)] = train["AVG_PRICE"].shift(i)
        
    
# Creating Week ago lags
j = 1
size = 2016
for i in range(size, size-65, -1):
    train["w_price_l_{}".format(j)] = train["DIFF_PRICE"].shift(i)
    train["w_demand_l_{}".format(j)] = train["RESIDUAL_DEMAND"].shift(i)
    train["w_avgPrice_l_{}".format(j)] = train["AVG_PRICE"].shift(i)
    j+=1

In [ ]:
# Drop NANS
train.dropna(inplace=True)
train.head(5)
train = train["2017":]

In [ ]:
#################### PROCESSING THE DATA FOR MLP NETWORK ###################

In [ ]:
########### THIS IS FOR MULTILAYER PERCEPTRON PURPOSES
train1 = data[['hour', 'weekday', 'month', 'business hour', 'public holiday']]
train1 = train1["2017":]

In [ ]:
# Scaling the categorical variables using the same scaler used for LSTM variables
cont = ['hour', 'weekday', 'month', 'business hour', 'public holiday']
for i in cont:
    train1[i] = feature_scaler.transform(pd.DataFrame(train1[i]))

In [ ]:
features1 = train1[train1.index.minute == 0]
features1 = features1[features1.index.hour == 0]

# Seperating training and test data for Multi-Layer Perceptron Network
features_train1 = features1[:'2018']
features_test1 = features1['2019':'2019-06-30']

# Reshaping the features and test data to NP-Array as per Keras input requirement
features_train1 = features_train1.to_numpy().reshape(features_train1.shape[0], features_train1.shape[1])
features_test1 = features_test1.to_numpy().reshape(features_test1.shape[0], features_test1.shape[1])

In [ ]:
#################### PROCESSING THE DATA FOR LSTM NETWORK ###################

In [ ]:
# create feature and label dataframes
prelim_features = train.drop(['RRP5MIN', 'RESIDUAL_DEMAND', 'AVG_PRICE', 'DIFF_PRICE', 'hour', 'weekday', 'month', 'business hour', 'public holiday'], axis=1)
prelim_labels = pd.DataFrame(train[['RRP5MIN']])

In [ ]:
# format labels to 24 hour output range
for i in range(0, 288):
    prelim_labels['t_{}'.format(i)] = prelim_labels['RRP5MIN'].shift(-i)
prelim_labels.drop(['RRP5MIN'], axis=1, inplace=True)

# apply one-day discretization to the data
labels = prelim_labels[prelim_labels.index.minute == 0]
labels = labels[labels.index.hour == 0]
features = prelim_features[prelim_features.index.minute == 0]
features = features[features.index.hour == 0]

features_train = features[:'2018']
features_test = features['2019':'2019-06-30']
labels_train = labels[:'2018']

samples_train = len(features_train)
samples_test = len(features_test)
timesteps = 335

# convert pandas data frames to numpy ndarrays
features_train = features_train.to_numpy().reshape(samples_train, timesteps, 3)
features_test = features_test.to_numpy().reshape(samples_test, timesteps, 3)
labels_train = labels_train.to_numpy()

# check for correct data shape
features_train.shape, labels_train.shape

In [ ]:
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
import json

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
import tensorflow as tf

In [ ]:
#############################################################################
########### CONCATENATE THE 2 NN & COMPILE THEM TO FORM BIGGER NN ##########
###########################################################################

In [ ]:
from keras.layers import concatenate

In [ ]:
# Creating the 2 models
mlp_shape = (features_train1.shape[1],)
lstm_shape = (None, features_train.shape[1], 3)

# mlp = cm.create_mlp((features_train1.shape[1],))
# lstm = cm.create_conv_lstm((None, features_train.shape[1], 3))      

mlp = cm.create_mlp(mlp_shape)
lstm = cm.create_conv_lstm(lstm_shape)

In [ ]:
# Merging the 2 networks into a bigger network 
combinedInput = concatenate([mlp.output, lstm.output])

In [ ]:
# Mapping the bigger Network to the output layer to predict one-day ahead i.e. 288 intervals
x = Dense(32, activation="relu")(combinedInput)
x = Dense(288, activation="sigmoid")(x)
model = Model(inputs=[mlp.input, lstm.input], outputs=x)

In [ ]:
# Compiling the model with Mean Absolute Error as loss function and Adam as optimizer
model.compile(loss='mse', optimizer='adam')

In [ ]:
results, hist = cm.train_predict_evaluate(model, features_train1, features_train, labels_train, 
                                       features_test1, features_test, X_test, X_test.index, label_scaler,
                                       32, 160)

In [ ]:
# Plot of predictions against Actuals
cm.plot_chart(results["2019-01-01":"2019-01-25"], legend=True)

In [ ]:
# Training loss comparision plot
cm.plot_chart(pd.DataFrame(hist.history), xlab='Training Epoch', ylab='Mean Squared Error', title='Training and Validation Error over the Course of Training', legend=True)

In [ ]:
# Quantifying Performance using MAE, MSE, RMSE
cm.quantify_performance(results)

In [ ]:
###########################################################
################## FINE TUNE THE MODEL ###################
##########################################################

In [ ]:
# Our function to find the work with different parameters
def fine_tune(mlp_dense1, conv1D_dense, lstm_dense, dense2, activation_func, 
             dropout_rate, loss, mlp_shape, lstm_shape):
    # Creating the 2 models
    mlp = cm.create_mlp(mlp_shape, mlp_dense1, dense2)
    lstm = cm.create_conv_lstm(lstm_shape, conv1D_dense, lstm_dense, dense2, 
                               activation_func, dropout_rate)
    
    # Merging the 2 networks into a bigger network 
    combinedInput = concatenate([mlp.output, lstm.output])
    
        # Mapping the bigger Network to the output layer to predict one-day ahead i.e. 288 intervals
    x = Dense(dense2, activation="relu")(combinedInput)
    x = Dense(288, activation="sigmoid")(x)
    model = Model(inputs=[mlp.input, lstm.input], outputs=x)
    
    # Compiling the model with Mean Absolute Error as loss function and Adam as optimizer
    model.compile(loss=loss, optimizer='adam')
    return model

In [ ]:
# We will try 39 different models with differen parameters, changing 1 thing at a time
# Aim is to find the best working parameters


models = []

#############################################
#
# Big Model
#
#############################################

# Different Activations
models.append(fine_tune(128, 200, 200, 64, 'sigmoid', 
             0.1, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(128, 200, 200, 64, 'relu', 
             0.1, 'mse', mlp_shape, lstm_shape))

# Different Dropouts #
models.append(fine_tune(128, 200, 200, 64, 'sigmoid', 
             0.2, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(128, 200, 200, 64, 'relu', 
             0.2, 'mse', mlp_shape, lstm_shape))

# Different loss Functions
models.append(fine_tune(128, 200, 200, 64, 'sigmoid', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(128, 200, 200, 64, 'sigmoid', 
             0.2, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(128, 200, 200, 64, 'relu', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(128, 200, 200, 64, 'relu', 
             0.2, 'mae', mlp_shape, lstm_shape))

#############################################
#
# MEDIUM MOEL
#
#############################################

# Different Activations
models.append(fine_tune(100, 128, 128, 50, 'sigmoid', 
             0.1, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(100, 128, 128, 50, 'relu', 
             0.1, 'mse', mlp_shape, lstm_shape))

# Different Dropouts #
models.append(fine_tune(100, 128, 128, 50, 'sigmoid', 
             0.2, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(100, 128, 128, 50, 'relu', 
             0.2, 'mse', mlp_shape, lstm_shape))

# Different loss Functions
models.append(fine_tune(100, 128, 128, 50, 'sigmoid', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(100, 128, 128, 50, 'sigmoid', 
             0.2, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(100, 128, 128, 50, 'relu', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(100, 128, 128, 50, 'relu', 
             0.2, 'mae', mlp_shape, lstm_shape))

#############################################
#
# SMALL MODEL
#
#############################################

# Different Activations
models.append(fine_tune(64, 128, 64, 32, 'sigmoid', 
             0.1, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(64, 128, 64, 32, 'relu', 
             0.1, 'mse', mlp_shape, lstm_shape))

# Different Dropouts #
models.append(fine_tune(64, 128, 64, 32, 'sigmoid', 
             0.2, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(64, 128, 64, 32, 'relu', 
             0.2, 'mse', mlp_shape, lstm_shape))

# Different loss Functions
models.append(fine_tune(64, 128, 64, 32, 'sigmoid', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 128, 64, 32, 'sigmoid', 
             0.2, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 128, 64, 32, 'relu', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 128, 64, 32, 'relu', 
             0.2, 'mae', mlp_shape, lstm_shape))

#############################################
#
# CURRENT MODEL
#
#############################################

# Different Activations
models.append(fine_tune(64, 64, 64, 32, 'sigmoid', 
             0.1, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(64, 64, 64, 32, 'relu', 
             0.1, 'mse', mlp_shape, lstm_shape))

# Different Dropouts #
models.append(fine_tune(64, 64, 64, 32, 'sigmoid', 
             0.2, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(64, 64, 64, 32, 'relu', 
             0.2, 'mse', mlp_shape, lstm_shape))

# Different loss Functions
models.append(fine_tune(64, 64, 64, 32, 'sigmoid', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 64, 64, 32, 'sigmoid', 
             0.2, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 64, 64, 32, 'relu', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 64, 64, 32, 'relu', 
             0.2, 'mae', mlp_shape, lstm_shape))

#############################################
#
# VARIATION OF CURRENT
#
#############################################

# Different Activations
models.append(fine_tune(64, 32, 32, 32, 'sigmoid', 
             0.1, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(64, 32, 32, 32, 'relu', 
             0.1, 'mse', mlp_shape, lstm_shape))

# Different Dropouts #
models.append(fine_tune(64, 32, 32, 32, 'sigmoid', 
             0.2, 'mse', mlp_shape, lstm_shape))
models.append(fine_tune(64, 32, 32, 32, 'relu', 
             0.2, 'mse', mlp_shape, lstm_shape))

# Different loss Functions
models.append(fine_tune(64, 32, 32, 32, 'sigmoid', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 32, 32, 32, 'sigmoid', 
             0.2, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 32, 32, 32, 'relu', 
             0.1, 'mae', mlp_shape, lstm_shape))
models.append(fine_tune(64, 32, 32, 32, 'relu', 
             0.2, 'mae', mlp_shape, lstm_shape))

In [ ]:
for i, model in enumerate(models):
    results, hist = cm.train_predict_evaluate(model, features_train1, features_train, labels_train, 
                                       features_test1, features_test, X_test, X_test.index, label_scaler,
                                       32, 160);
    print("Model {}".format(i))
    cm.quantify_performance(results)
    print("--------------------------------------------")
    print("--------------------------------------------")

In [ ]:
###########################################################
################ TRAIN ON ALL THE DATA ###################
################ USING MODEL 27(BEST) ###################
########################################################

In [ ]:
mlp = cm.create_mlp(mlp_shape, 64, 32)
lstm = cm.create_conv_lstm(lstm_shape, 128, 64, 32, 'relu', 0.1)

combinedInput = concatenate([mlp.output, lstm.output])

x = Dense(32, activation="relu")(combinedInput)
x = Dense(288, activation="sigmoid")(x)
model = Model(inputs=[mlp.input, lstm.input], outputs=x)

model.compile(loss='mse', optimizer='adam')

In [ ]:
results, hist = cm.train_predict_evaluate(model, features_train1, features_train, labels_train, 
                                       features_test1, features_test, X_test, X_test.index, label_scaler,
                                       32, 160)

In [ ]:
cm.plot_chart(results["2019-01-01":"2019-01-25"], legend=True)

In [ ]:
cm.plot_chart(pd.DataFrame(hist.history), xlab='Training Epoch', ylab='Mean Squared Error', title='Training and Validation Error over the Course of Training', legend=True)

In [ ]:
cm.quantify_performance(results)

In [ ]:
# Input prices into optimizer
import optimizer_module as om
numDays = 100 # Number of days to run model
start = 0 # Starting time interval from price data
bStorage0 = 0 # Starting battery charge

predPrices = results.iloc[start:start+(numDays*288)]["prediction"].tolist()
realPrices = results.iloc[start:start+(numDays*288)]["true values"].tolist()
outputResults = 1
outputActions = 1

print("Optimizer results for real prices")
realNxtAction, realNxtBatCharge, realActions = om.optimize(realPrices, bStorage0, outputResults, outputActions)

print("\nOptimizer results for predicted prices\nProfit is incorrect as it is calculating predicted profit not actual profit")
predNxtAction, predNxtBatCharge, predActions = om.optimize(predPrices, bStorage0, outputResults, outputActions)

maxProfit = sum([realActions[i]*realPrices[i]/12 for i in range(numDays*288)])
actualProfit = sum([predActions[i]*realPrices[i]/12 for i in range(numDays*288)])

print("\n----------RESULTS----------")
print("Max profit possible: $%.4g" % (maxProfit))
print("Actual profit: $%.4g -> %.4g%% of max profit possible" % (actualProfit,actualProfit/maxProfit*100))    

In [ ]:
%reload_ext autoreload

In [ ]:
%load_ext autoreload
%autoreload 2